In [1]:
import os
import gc
import pickle
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
import cudf

pd.options.display.max_rows = None
pd.options.display.max_columns = None

%matplotlib inline

In [2]:
df = pd.read_csv('features.csv')

In [3]:
df.head()

,customer_ID,S_2,D_39,D_41,D_42,D_43,D_44,D_45,D_46,D_47,D_48,D_49,D_50,D_51,D_52,D_53,D_54,D_55,D_56,D_58,D_59,D_60,D_61,D_62,D_63,D_64,D_65,D_66,D_68,D_69,D_70,D_71,D_72,D_73,D_74,D_75,D_76,D_77,D_78,D_79,D_80,D_81,D_82,D_83,D_84,D_86,D_87,D_88,D_89,D_91,D_92,D_93,D_94,D_96,D_102,D_103,D_104,D_105,D_106,D_107,D_108,D_109,D_110,D_111,D_112,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,D_130,D_131,D_132,D_133,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27,P_2,P_3,P_4,B_1,B_2,B_3,B_4,B_5,B_6,B_7,B_8,B_9,B_10,B_11,B_12,B_13,B_14,B_15,B_16,B_17,B_18,B_19,B_20,B_21,B_22,B_23,B_24,B_25,B_26,B_27,B_28,B_29,B_30,B_31,B_32,B_33,B_36,B_37,B_38,B_39,B_40,B_41,B_42,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9,R_10,R_11,R_12,R_13,R_14,R_15,R_16,R_17,R_18,R_19,R_20,R_21,R_22,R_23,R_24,R_25,R_26,R_27,R_28
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,0.008376,0.007045,NaN,0.355097,0.381721,0.232541,0.749022,0.931925,0.636798,NaN,NaN,0.008505,0.257918,NaN,1.009888,0.678041,NaN,0.673155,0.279011,0.001999,0.845026,0.112573,CO,R,0.003677,NaN,6.0,0.002893,0.255467,0.013176,0.007019,NaN,0.362555,0.338296,NaN,NaN,0.002008,0.009383,0.406832,0.008304,NaN,0.005456,0.008112,0.009082,NaN,NaN,0.007533,0.009429,0.006783,0.004019,0.002943,0.001115,0.004340,0.001371,0.007033,NaN,NaN,0.002144,NaN,0.006973,NaN,NaN,1.003346,0.003928,1.0,0.464769,0.0,-1.0,0.461045,0.459714,0.0,0.727194,0.428788,0.009761,0.365854,0.008233,1.0,0.009708,1.005559,0.007325,0.007439,0.007083,NaN,0.008561,NaN,NaN,NaN,NaN,NaN,0.008365,0.008801,0.001185,NaN,0.006018,0.009393,0.003585,0.401315,0.002239,0.008523,0.327650,0.127948,NaN,0.048080,0.184423,0.005208,0.408499,0.006978,0.008038,0.003621,0.004491,0.001774,0.853806,0.131347,0.868075,0.972705,0.004930,0.007958,0.523405,0.480283,0.001637,0.006680,1.000940,0.009355,0.901268,0.020332,0.034404,0.182016,0.007657,0.004749,0.027181,0.008233,0.154105,0.008163,0.006533,0.007149,0.423158,NaN,0.407278,0.109469,0.004621,0.005013,0.006692,0.184154,0.009886,0.003749,0.006558,0.004967,0.521413,NaN,0.0,1,0.000358,1.004676,0.000329,0.003749,3.0,NaN,0.330606,0.005743,NaN,0.003428,0.005751,0.507771,0.000728,0.008425,0.008175,0.002113,0.004826,NaN,0.007801,0.005049,1.000701,0.000934,0.001050,0.002937,0.005534,0.002551,0.009574,0.006862,0.007049,0.008520,0.005952,0.008763,0.000349,0.005358,NaN,1.000625,0.001186
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,0.002836,0.000565,NaN,0.312375,0.134045,0.232197,0.416163,0.939839,0.484000,NaN,0.006556,0.006998,0.253597,NaN,1.008957,0.611879,NaN,0.567915,0.275475,0.009932,0.842110,0.077126,CO,R,0.008699,NaN,6.0,0.005307,0.007638,0.012317,0.003940,NaN,0.290779,0.275103,NaN,0.072436,0.000811,0.009827,0.608082,0.008636,0.005899,0.009630,0.006017,0.001705,NaN,NaN,0.001632,0.001945,0.008371,0.002443,0.002520,0.005529,0.006890,0.009558,0.008599,NaN,NaN,0.008818,NaN,0.007328,NaN,NaN,1.002003,0.006697,1.0,0.477699,0.0,-1.0,0.461896,0.465308,0.0,0.732167,0.576645,0.004936,0.365156,0.008392,1.0,0.008002,1.002762,1.004064,0.004804,0.000823,NaN,0.000696,NaN,NaN,NaN,NaN,NaN,0.006429,0.000438,0.004144,NaN,0.002282,0.003261,0.002704,0.394459,0.004846,0.008937,0.322127,0.008656,NaN,0.289070,0.184495,0.004601,0.509170,0.002461,0.004669,0.004006,0.004215,0.004033,0.303755,0.140477,0.085234,0.969171,0.009193,0.005511,0.533376,0.522688,0.003317,0.007121,0.814569,0.011885,0.901686,0.009887,0.046754,0.328685,0.007138,0.008674,0.016825,0.002511,0.057497,0.012143,0.003472,0.004912,0.426391,NaN,0.407492,0.209688,0.009138,0.006126,0.005744,0.340210,0.002182,0.007864,0.005851,0.004641,0.505322,NaN,0.0,1,0.008105,1.005377,0.004619,0.003541,3.0,NaN,0.282520,0.008473,NaN,0.001009,0.005574,0.408810,0.005981,0.003779,0.000730,0.007040,0.003869,NaN,0.009881,0.007829,1.003468,0.009944,0.002582,0.000130,0.005474,0.006805,0.008637,0.000246,0.006

In [32]:
window = 5
_df = df[['customer_ID', 'S_2', 'D_39']].copy()

_df = cudf.from_pandas(_df)

def func(x):
    n = 0
    for i in range(window):
        n += x.shift(i) / window
    return n

# func = lambda x: (x.shift(0) + x.shift(1) + x.shift(2)) / 3

_df['feats_D_39'] = _df.groupby('customer_ID')['D_39'].pipe(func)

In [33]:
_df.head()

,customer_ID,S_2,D_39,feats_D_39
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,0.008376,<NA>
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,0.002836,<NA>
2,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-05-11,0.005452,<NA>
3,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-06-26,0.008479,<NA>
4,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-07-27,0.001563,0.005341076


In [9]:
(0.008479 + 0.001563 + 0.005452)/3

0.005164666666666667

In [4]:
# df.groupby(['customer_ID']).rolling(window=2)['D_39'].mean().reset_index()
df['fe_rollingMean_D_39'] = df.groupby('customer_ID')['D_39'].transform(lambda x: x.rolling(2).mean())

_df = df.groupby('customer_ID').tail(1).reset_index(drop=True)

In [5]:
_df[['customer_ID', 'S_2', 'D_39', 'fe_rollingMean_D_39']].head()

,customer_ID,S_2,D_39,fe_rollingMean_D_39
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2018-03-29,0.007839,0.006902
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,2018-03-10,0.002275,0.001970
2,0229ecb6a6eb8a724637bd7d8b988d06562f2441f5112e...,2018-03-20,0.000656,0.121753
3,0264ec10c09b97662a7b19871e4eaf540050afccf65d26...,2018-03-19,0.007139,0.004190
4,02f887d9280480f14f8cfe1650661f91b034f087f056b8...,2018-03-21,0.240051,0.461136


In [6]:
feats = ['D_39', 'D_41']

rename_list = [f'fe_rollingMean_{f}' for f in feats]

df[rename_list] = df.groupby('customer_ID')[feats].transform(lambda x: x.rolling(2).mean())

_df = df.groupby('customer_ID').tail(1).reset_index(drop=True)

In [7]:
df[rename_list].head()

,fe_rollingMean_D_39,fe_rollingMean_D_41
0,NaN,NaN
1,0.005606,0.003805
2,0.004144,0.002549
3,0.006965,0.004968
4,0.005021,0.003781


In [8]:
kmeans = KMeans(n_clusters=8)

In [19]:
feats = ['R_3', 'R_1']

kmeans = kmeans.fit(df[feats])

a = kmeans.predict(df[feats])
b = kmeans.transform(df[feats])

In [20]:
a

array([3, 3, 3, ..., 0, 0, 0], dtype=int32)

In [22]:
b[0]

array([0.47521348, 0.55065966, 1.10782913, 0.23212067, 0.6740886 ,
       0.29785597, 1.64150931, 1.25625403])

In [23]:
b = pd.DataFrame(b, columns=[np.arange(8)])

In [25]:
b.head()

,0,1,2,3,4,5,6,7
0,0.475213,0.550660,1.107829,0.232121,0.674089,0.297856,1.641509,1.256254
1,0.376319,0.579694,1.078037,0.133475,0.619393,0.396716,1.629673,1.349918
2,0.174069,0.677728,1.039569,0.070442,0.543327,0.598833,1.619611,1.542247
3,0.174772,0.676316,1.038462,0.069492,0.542336,0.598012,1.618415,1.541164
4,0.168899,0.678343,1.036094,0.074987,0.539316,0.603725,1.616667,1.546176


In [26]:
b.shape

(5569, 8)